### RPI3B pinout

<div style="text-align:center;"><img src="images/rpi3bplus_pinout.png"  width="300"/></div>
<br/>

<div style="text-align:center;"><img src="images/pwm_servomotor.png"  width="300"/></div>

In [3]:
!sudo usermod -a -G gpio banshee

[sudo] password for banshee: 


## Library RPI.GPIO  

Main library searching for PWM control with RPI (an important bug described bellow)
<https://sourceforge.net/p/raspberry-gpio-python/wiki/TechRef/>


## Test 1 PWM

Basic setting freq and cycle.

In [1]:
import RPi.GPIO as GPIO
import time

print('Version : ' + GPIO.VERSION)

servoPIN = 18
GPIO.setmode(GPIO.BCM)
GPIO.setup(servoPIN, GPIO.OUT)
p = GPIO.PWM(servoPIN, 50) # GPIO 18 for PWM with 10Hz
p.start(75) 
try:
    time.sleep(1000)
except KeyboardInterrupt:
    None

p.stop()
GPIO.cleanup()
print('Cleaned')

Version : 0.7.0
Cleaned


### **BUG** RPI.GPIO

Something weird is happening here. The first the program runs well, generating a wave with desired freq. In the second execution and avove (watching with oscilloscope) the weave has a **high freq (300-800Hz)**. Appears to be a bug with the **RPI.GPIO** library and **stop** function as I found at <https://sourceforge.net/p/raspberry-gpio-python/tickets/94/> and <https://forums.raspberrypi.com/viewtopic.php?t=277137>.

After a kernel restart the programs runs well.

<code>
If I remember correctly there was a bug in the RPi.GPIO code that meant once you use p.stop() as in your code the PWM wont work again till you reboot your pi.
I guess this never got fixed. Instead of stopping the PWM with p.stop() just set a p.ChangeDutyCycle(0) that will stop the servo from moving and you can then set another value when you want to start it moving again.
</code>
<br/

Alternative Version possibly fixed problem:
<https://github.com/wuestkamp/raspberry-gpio-python>


A **workaround** in production environment using RPI.GPIO is not stop GPIO.PWM, simply set DutyCycle to 0.

## Test 2 PWM

Changing frequency on running program

In [1]:
import RPi.GPIO as GPIO
import time

print('Version : ' + GPIO.VERSION)

servoPIN = 18
GPIO.setmode(GPIO.BCM)
GPIO.setup(servoPIN, GPIO.OUT)
p = GPIO.PWM(servoPIN, 50) # GPIO 18 for PWM with 50Hz
p.start(75) 
try:
    p.ChangeFrequency(10)
    time.sleep(10)
    p.ChangeFrequency(20)
    time.sleep(10)
    p.ChangeFrequency(30)
    time.sleep(10)
    p.ChangeFrequency(40)
    time.sleep(10)
    p.ChangeFrequency(50)
    time.sleep(10)
except KeyboardInterrupt:
    None

p.stop()
GPIO.cleanup()
print('Cleaned')

Version : 0.7.0
Cleaned


**Works as expected**

In [3]:
import RPi.GPIO as GPIO
import time

servoPIN = 18
GPIO.setmode(GPIO.BCM)
GPIO.setup(servoPIN, GPIO.OUT)

p = GPIO.PWM(servoPIN, 50) # GPIO 18 for PWM with 50Hz
p.start(2.5) 
try:
    p.ChangeDutyCycle(5)
    time.sleep(10)
    p.ChangeDutyCycle(20)
    time.sleep(10)
    p.ChangeDutyCycle(50)
    time.sleep(10)
    p.ChangeDutyCycle(65)
    time.sleep(10)
    p.ChangeDutyCycle(75)
    time.sleep(10)
    p.ChangeDutyCycle(90)
    time.sleep(10)
    p.ChangeDutyCycle(100)
    time.sleep(10)
except KeyboardInterrupt:
    None
    
p.stop()
GPIO.cleanup()

### ***FINAL NOTE***

The previously mentioned bug could be workarounded in production not stoping PWM,. In jupyter notebooks it's uncomfortable to develop short test with start/stop/cleanup code blocks.

For the **BanRC01** I'm using Servo Kit from Adadruit with **PCA9685** so I'm not use RPI.GPIO library, so this not be a problem.

## Using Servos

General Documentation specifies times as
<div style="text-align:center;"><img src="images/pwm_servomotor.png"  width="300"/></div>

and others as

<div style="text-align:center;"><img src="images/servo-control.png"  width="300"/></div>
<br/>

So it's neccesary calibrate and read datasheets for every motor for identify the rights duty cycles.

Read about at : <https://github.com/gavinlyonsrepo/RpiMotorLib/blob/master/Documentation/Servo_RPI_GPIO.md>

```
Given a 50 Hz frequency we can calculate the required duty cycle for any pulse width. For example: We need a 1.5 ms pulse to center the servo, or a Duty Cycle = 0.0015 * 50 = 0.075 (i.e 7.5%). Similarly, 1 ms pulse (- 90 degrees or 0 degrees) requires a Duty Cycle = 0.001 * 50 = 5% 2 ms pulse (+ 90 degrees or 180), Duty Cycle = 0.002 * 50 = 10% Thus the duty cycle range should be from 5 - 10% with the center at 7.5%.

Every servo is different, so you will need to calibrate it for the best performance. It was found 7.5 for center , 11 for max duty percentage or left position (180 degress) and 2 for min duty percentage or right postion (0 degrees) Check datasheet for recommend pulse width and calibrate accordingly.
```


Read about at : <http://www.robotplatform.com/knowledge/servo/servo_control_tutorial.html>

```
We have reached the most important and interesting section of this tutorial: Servo Control.

Controlling a servo as discussed before involves sending a modulated square-wave pulse which is known as pulse-Width-Modulation (PWM) or Pulse-Duration-Modulation (PDM). If signal voltage from peak to peak (amplitude) is taken care as per the datasheet (which is generally 3V to 5V), then there two other main factors to be considered while sending a PWM signal to servo; “Frequency” and “Duty cycle”.

Frequency

Servo expects continuous pulses to move or hold its position. Frequency (or repetition time, or cycle time) is the number of times a positive pulse is fed to servo in a unit time (which is generally measured in seconds); for analog servos, frequency is generally 50 times a second (50Hz) and for digital servos it is 300 times a second (300Hz).

50Hz = Positive pulse 50 times a second; i.e. 1/50 = 0.02 seconds = 20ms (timeout period). This means every 20 milliseconds servo expects a pulse to retain its corresponding angular position.

300Hz = Positive pulse 300 times a second; i.e. 1/300 = 0.0033 seconds = 3.33ms (timeout period). This means every ~3 milliseconds servo expects a pulse to retain its corresponding angular position.

If servo does not receive a pulse before the timeout period, then servo releases its hold and can move to any forced position.

Note: Required pulse frequency for few servos may be less, or more depending on the particular model and manufacturer.

Duty Cycle

“Duty cycle” is the width of positive pulse (square wave) and a deciding factor for servo’s angular position. For example, if you have a servo with 180° turn, then 90° is the center position of the servo with 0° being minimum, and 180°, being the maximum. Now, if a positive pulse of 1.5ms is sent, then the servo stays at 90° (servo center) as long as it receives the same pulse. If another pulse of 1ms is sent, the circuit tries to move the shaft to 0°, and a pulse of 2ms tries to move the output shaft to 180°.  This means, a pulse shorter than 1.5ms moves the servo in one direction and wider than 1.5ms moves it in another direction.

Different servo models have different minimum and maximum pulse requirements. For example, a Hextronik servo I have has a minimum pulse requirement of 0.5ms to move to 0° and maximum pulse duration of 2.5ms to move to 180°. Sending a pulse of 1ms moves it to 45° and 2ms moves it to 135°. Another servo requires 1ms pulse to move to 0°, 1.5ms to move to 45° and 2ms to move to 90° and maximum angular rotation being 90°. The de-facto standard is 1ms for minimal angle, 1.5ms for servo center and 2ms for maximum angle. Servo center is almost always 1.5ms and minimum and maximum should be verified in product’s datasheet.
```


Expressing duty as percentage of PWM for tipical case (50Hz) whe have:

```
Freq = 50Hz --> 1 cycle every 20 ms

# Center (1.5 ms)
20  ms --- 100
1.5 ms   --- x

x = 1.5*100 / 20 = 7.5%


# Left (1 ms)
20  ms --- 100
1   ms   --- x

x = 1*100 / 20 = 5%


# Right (2 ms)
20 ms --- 100
2 ms  --- x

x = 2*100 / 20 = 10%
```


#### **Arduino** Servo lib defines this generic limits

```
Servo - attach()

Attach the Servo variable to a pin. Note that in Arduino 0016 and earlier, the Servo library supports servos on only two pins: 9 and 10.
Syntax

servo.attach(pin) 
servo.attach(pin, min, max)

Parameters

    servo: a variable of type Servo
    pin: the number of the pin that the servo is attached to
    min (optional): the pulse width, in microseconds, corresponding to the minimum (0 degree) angle on the servo (defaults to 544)
    max (optional): the pulse width, in microseconds, corresponding to the maximum (180 degree) angle on the servo (defaults to 2400)

```

In [ ]:
import RPi.GPIO as GPIO
import time

print('Version : ' + GPIO.VERSION)

servoPIN = 18
GPIO.setmode(GPIO.BCM)
GPIO.setup(servoPIN, GPIO.OUT)
servo1 = GPIO.PWM(servoPIN, 50) # GPIO 18 for PWM with 50Hz 

try:
    servo1.start(7.5)
    print ("Centering servo")
    time.sleep(2)
except KeyboardInterrupt:
    None

servo1.stop()
GPIO.cleanup()
print('Cleaned')

In [2]:
import RPi.GPIO as GPIO
import time

print('Version : ' + GPIO.VERSION)

servoPIN = 18
GPIO.setmode(GPIO.BCM)
GPIO.setup(servoPIN, GPIO.OUT)
servo1 = GPIO.PWM(servoPIN, 50) # GPIO 18 for PWM with 50Hz 

try:
    #start PWM running, but with value of 0 (pulse off)
    servo1.start(7.5)
    print ("Waiting for 2 seconds")
    time.sleep(2)

    #Let's move the servo!
    print ("Rotating 180 degrees in 10 steps")

    # Define variable duty
    duty = 2

    # Loop for duty values from 2 to 12 (0 to 180 degrees)
    while duty <= 6:
        print('Set Duty : '+str(duty))
        servo1.ChangeDutyCycle(duty)
        time.sleep(0.1)
        duty = duty + 1
        
    # Wait a couple of seconds
    time.sleep(2)

    # Turn back to 90 degrees
    print ("Turning back to 90 degrees for 2 seconds")
    servo1.ChangeDutyCycle(7.5)
    time.sleep(2)

    #turn back to 0 degrees
    print ("Turning back to 0 degrees")
    servo1.ChangeDutyCycle(2)
    time.sleep(0.5)
    servo1.ChangeDutyCycle(0)
except KeyboardInterrupt:
    None

servo1.stop()
GPIO.cleanup()
print('Cleaned')

Version : 0.7.0
Waiting for 2 seconds
Rotating 180 degrees in 10 steps
Set Duty : 2
Set Duty : 3
Set Duty : 4
Set Duty : 5
Set Duty : 6
Turning back to 90 degrees for 2 seconds
Turning back to 0 degrees
Cleaned


In [1]:
import RPi.GPIO as GPIO
import time

print('Version : ' + GPIO.VERSION)

servoPIN = 18
GPIO.setmode(GPIO.BCM)
GPIO.setup(servoPIN, GPIO.OUT)
servo1 = GPIO.PWM(servoPIN, 50) # GPIO 18 for PWM with 50Hz 

try:
    #start PWM running, but with value of 0 (pulse off)
    servo1.start(0)
    print ("Waiting for 2 seconds")
    time.sleep(2)

    #Let's move the servo!
    print ('Center')
    duty = 7.5
    servo1.ChangeDutyCycle(duty)
    time.sleep(2)

    
    # Moving to left
    while duty >= 5:
        print('Set Duty : '+str(duty))
        servo1.ChangeDutyCycle(duty)
        time.sleep(1)
        duty = duty - 0.5
        
    duty = 5
           
    # Moving to left
    while duty <= 10:
        print('Set Duty : '+str(duty))
        servo1.ChangeDutyCycle(duty)
        time.sleep(1)
        duty = duty + 0.5
        
    # Wait a couple of seconds
    time.sleep(2)

    # Turn back to 90 degrees
    print ("Turning back to 90 degrees for 2 seconds")
    servo1.ChangeDutyCycle(7.5)
    time.sleep(2)
except KeyboardInterrupt:
    None

servo1.stop()
GPIO.cleanup()
print('Cleaned')

Version : 0.7.0
Waiting for 2 seconds
Center
Set Duty : 7.5
Set Duty : 7.0
Set Duty : 6.5
Set Duty : 6.0
Set Duty : 5.5
Set Duty : 5.0
Set Duty : 5
Set Duty : 5.5
Set Duty : 6.0
Set Duty : 6.5
Set Duty : 7.0
Set Duty : 7.5
Set Duty : 8.0
Set Duty : 8.5
Set Duty : 9.0
Set Duty : 9.5
Set Duty : 10.0
Turning back to 90 degrees for 2 seconds
Cleaned


In [1]:
import RPi.GPIO as GPIO
import time
import ipywidgets as widgets

servoPIN = 18
GPIO.setmode(GPIO.BCM)
GPIO.setup(servoPIN, GPIO.OUT)
servo1 = GPIO.PWM(servoPIN, 50) # GPIO 18 for PWM with 50Hz 
servo1.start(0)

style = {'description_width': 'initial'}
servo_duty_widget = widgets.FloatSlider(description='Servo Position', style=style, min=0, max=15, step=0.1, value=0, layout=widgets.Layout(width='50%', height='30px'))
output = widgets.Output()

def on_servo_duty_value_change(change):
    #with output:
        duty = change['new']
        print('Set Duty : '+str(duty))
        servo1.ChangeDutyCycle(duty)
        
servo_duty_widget.observe(on_servo_duty_value_change, names='value')

engine_widget = widgets.VBox([
    servo_duty_widget
])
display(engine_widget)

In [ ]:
servo1.stop()
GPIO.cleanup()
print('Cleaned')